<a href="https://colab.research.google.com/github/fralfaro/MAT281_2022/blob/main/labs/lab_08/lab_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# MAT281 - Laboratorio N°08



<a id='p1'></a>
## I.- Problema 01


<img src="https://freedesignfile.com/upload/2013/06/Car-logos-1.jpg" width="360" height="360" align="center"/>


El conjunto de datos se denomina `vehiculos_procesado_con_grupos.csv`, el cual contine algunas de las características más importante de un vehículo.

En este ejercicio se tiene como objetivo, es poder clasificar los distintos vehículos basados en las cracterísticas que se presentan a continuación. La dificultad de este ejercicio radíca en que ahora tenemos variables numéricas y variables categóricas.

Lo primero será cargar el conjunto de datos:

In [18]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mean

from sklearn.preprocessing import MinMaxScaler
from sklearn.dummy import DummyClassifier
from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale


%matplotlib inline
sns.set_palette("deep", desat=.6)
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [3]:
# cargar datos
df = pd.read_csv("https://raw.githubusercontent.com/fralfaro/MAT281_2022/main/labs/lab_08/data/vehiculos_procesado_con_grupos.csv", sep=",")\
       .drop(
            ["fabricante", 
             "modelo",
             "transmision", 
             "traccion", 
             "clase", 
             "combustible",
             "consumo"], 
    
          axis=1)

df

,year,desplazamiento,cilindros,co2,clase_tipo,traccion_tipo,transmision_tipo,combustible_tipo,tamano_motor_tipo,consumo_tipo,co2_tipo,consumo_litros_milla
0,1984,2.5,4.0,522.764706,Vehículos Especiales,dos,Automatica,Normal,pequeño,alto,alto,0.222671
1,1984,4.2,6.0,683.615385,Vehículos Especiales,dos,Automatica,Normal,grande,muy alto,muy alto,0.291185
2,1985,2.5,4.0,555.437500,Vehículos Especiales,dos,Automatica,Normal,pequeño,alto,alto,0.236588
3,1985,4.2,6.0,683.615385,Vehículos Especiales,dos,Automatica,Normal,grande,muy alto,muy alto,0.291185
4,1987,3.8,6.0,555.437500,Coches Medianos,dos,Automatica,Premium,grande,alto,alto,0.236588
...,...,...,...,...,...,...,...,...,...,...,...,...
36786,2015,1.0,3.0,244.000000,Coches pequeños,dos,Automatica,Premium,muy pequeño,muy bajo,muy bajo,0.105150
36787,2016,0.9,3.0,246.000000,Coches pequeños,dos,Automatica,Premium,muy pequeño,muy bajo,muy bajo,0.108155
36788,2016,0.9,3.0,255.000000,Coches pequeños,dos,Manual,Premium,muy pequeño,muy bajo,muy bajo,0.111336
36789,2017,0.9,3.0,248.000000,Coches pequeños,dos,Automatica,Premium,muy pequeño,muy bajo,muy bajo,0.108155


En este caso, no solo se tienen datos numéricos, sino que también categóricos. Además, tenemos problemas de datos **vacíos (Nan)**. Así que para resolver este problema, seguiremos varios pasos:

### 1.- Normalizar datos

1. Cree un conjunto de datos con las variables numéricas, además, para cada dato vacía, rellene con el promedio asociado a esa columna. Finalmente, normalize los datos mediante el procesamiento **MinMaxScaler** de **sklearn**.

2.-  Cree un conjunto de datos con las variables categóricas , además, transforme de variables categoricas a numericas ocupando el comando **get_dummies** de pandas ([referencia](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)). Explique a grande rasgo como se realiza la codificación de variables numéricas a categóricas.

3.- Junte ambos dataset en uno, llamado **df_procesado**. 


In [28]:
df.isnull().sum() 

year                    0
desplazamiento          2
cilindros               3
co2                     0
clase_tipo              0
traccion_tipo           0
transmision_tipo        0
combustible_tipo        0
tamano_motor_tipo       2
consumo_tipo            0
co2_tipo                0
consumo_litros_milla    0
dtype: int64

In [29]:
# 1)

df1 = df[["year","desplazamiento","cilindros","co2","consumo_litros_milla"]]
df1["desplazamiento"].fillna(df1["desplazamiento"].mean(skipna=True), inplace=True)
df1["cilindros"].fillna(df1["cilindros"].mean(skipna=True), inplace=True)
df1.isnull().sum()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


year                    0
desplazamiento          0
cilindros               0
co2                     0
consumo_litros_milla    0
dtype: int64

In [30]:
scaler = MinMaxScaler()
scaler.fit(df1)
df1_normalized = pd.DataFrame(scaler.transform(df1), columns = ["year","desplazamiento","cilindros","co2","consumo_litros_milla"])
print(df1_normalized)

           year  desplazamiento  cilindros       co2  consumo_litros_milla
0      0.000000        0.243590   0.142857  0.398014              0.331027
1      0.000000        0.461538   0.285714  0.527672              0.475113
2      0.029412        0.243590   0.142857  0.424351              0.360294
3      0.029412        0.461538   0.285714  0.527672              0.475113
4      0.088235        0.410256   0.285714  0.424351              0.360294
...         ...             ...        ...       ...                   ...
36786  0.911765        0.051282   0.071429  0.173307              0.083878
36787  0.941176        0.038462   0.071429  0.174919              0.090196
36788  0.941176        0.038462   0.071429  0.182174              0.096886
36789  0.970588        0.038462   0.071429  0.176532              0.090196
36790  0.970588        0.038462   0.071429  0.183786              0.096886

[36791 rows x 5 columns]


In [31]:
# 2)

df2 = df[['clase_tipo',	'traccion_tipo', 'transmision_tipo',	'combustible_tipo',	'tamano_motor_tipo',	'consumo_tipo',	'co2_tipo']]
df2_dummed = pd.get_dummies(df2, prefix=['clase_tipo', 'traccion_tipo','transmision_tipo',	'combustible_tipo',	'tamano_motor_tipo',	'consumo_tipo',	'co2_tipo'])

In [32]:
# 3)

df_procesado = pd.concat([df1_normalized, df2_dummed], axis=1)
df_procesado

,year,desplazamiento,cilindros,co2,consumo_litros_milla,clase_tipo_Camionetas,clase_tipo_Coche Familiar,clase_tipo_Coches Grandes,clase_tipo_Coches Medianos,clase_tipo_Coches pequeños,...,consumo_tipo_alto,consumo_tipo_bajo,consumo_tipo_moderado,consumo_tipo_muy alto,consumo_tipo_muy bajo,co2_tipo_alto,co2_tipo_bajo,co2_tipo_moderado,co2_tipo_muy alto,co2_tipo_muy bajo
0,0.000000,0.243590,0.142857,0.398014,0.331027,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
1,0.000000,0.461538,0.285714,0.527672,0.475113,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0.029412,0.243590,0.142857,0.424351,0.360294,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
3,0.029412,0.461538,0.285714,0.527672,0.475113,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,0.088235,0.410256,0.285714,0.424351,0.360294,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36786,0.911765,0.051282,0.071429,0.173307,0.083878,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
36787,0.941176,0.038462,0.071429,0.174919,0.090196,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
36788,0.941176,0.038462,0.071429,0.182174,0.096886,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
36789,0.970588,0.038462,0.071429,0.176532,0.090196,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1


### 2.- Realizar ajuste mediante kmeans

Una vez depurado el conjunto de datos, es momento de aplicar el algoritmo de **kmeans**.

1. Ajuste el modelo de **kmeans** sobre el conjunto de datos, con un total de **8 clusters**.
2. Asociar a cada individuo el correspondiente cluster y calcular valor de los centroides de cada cluster.
3. Realizar un resumen de las principales cualidades de cada cluster. Para  esto debe calcular (para cluster) las siguientes medidas de resumen:
    * Valor promedio de las variables numérica
    * Moda para las variables numericas
    
    

In [34]:
# 1)

X = np.array(df_procesado)
kmeans = KMeans(n_clusters=8,n_init=25, random_state=7)
kmeans.fit(X)

KMeans(n_init=25, random_state=7)

In [35]:
# 2)

centroids = kmeans.cluster_centers_ 
clusters = kmeans.labels_ 

df_procesado["cluster"] = clusters
df_procesado

,year,desplazamiento,cilindros,co2,consumo_litros_milla,clase_tipo_Camionetas,clase_tipo_Coche Familiar,clase_tipo_Coches Grandes,clase_tipo_Coches Medianos,clase_tipo_Coches pequeños,...,consumo_tipo_bajo,consumo_tipo_moderado,consumo_tipo_muy alto,consumo_tipo_muy bajo,co2_tipo_alto,co2_tipo_bajo,co2_tipo_moderado,co2_tipo_muy alto,co2_tipo_muy bajo,cluster
0,0.000000,0.243590,0.142857,0.398014,0.331027,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5
1,0.000000,0.461538,0.285714,0.527672,0.475113,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,0.029412,0.243590,0.142857,0.424351,0.360294,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5
3,0.029412,0.461538,0.285714,0.527672,0.475113,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,0.088235,0.410256,0.285714,0.424351,0.360294,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36786,0.911765,0.051282,0.071429,0.173307,0.083878,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,2
36787,0.941176,0.038462,0.071429,0.174919,0.090196,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,2
36788,0.941176,0.038462,0.071429,0.182174,0.096886,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,4
36789,0.970588,0.038462,0.071429,0.176532,0.090196,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,2


In [36]:
# 3)

df_procesado.groupby(['cluster']).describe()

year                                                              \
          count      mean       std       min       25%       50%       75%   
cluster                                                                       
0        4108.0  0.384236  0.280024  0.000000  0.147059  0.323529  0.617647   
1        6204.0  0.409209  0.300554  0.000000  0.147059  0.352941  0.676471   
2        3857.0  0.695702  0.293674  0.000000  0.529412  0.823529  0.911765   
3        3968.0  0.588065  0.252886  0.029412  0.404412  0.617647  0.794118   
4        3596.0  0.438404  0.328144  0.000000  0.117647  0.382353  0.764706   
5        4677.0  0.476852  0.288492  0.000000  0.205882  0.500000  0.705882   
6        3036.0  0.452443  0.270127  0.000000  0.205882  0.470588  0.676471   
7        7345.0  0.524326  0.297157  0.000000  0.264706  0.558824  0.794118   

             desplazamiento            ... co2_tipo_muy alto       \
         max          count      mean  ...               75%  max   
cluster                                ...                          
0        1.0         4108.0  0.585590  ...               1.0  1.0   
1        1.0         6204.0  0.333730  ...               0.0  0.0   
2        1.0         3857.0  0.179404  ...               0.0  0.0   
3        1.0         3968.0  0.362121  ...               0.0  0.0   
4        1.0         3596.0  0.159576  ...               0.0  0.0   
5        1.0         4677.0  0.450916  ...               0.0  1.0   
6        1.0         3036.0  0.562730  ...               1.0  1.0   
7        1.0         7345.0  0.250778  ...               0.0  0.0   

        co2_tipo_muy bajo                                               
                    count      mean       std  min  25%  50%  75%  max  
cluster                                                                 
0                  4108.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  
1                  6204.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  
2                  3857.0  0.982110  0.132567  0.0  1.0  1.0  1.0  1.0  
3                  3968.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  
4                  3596.0  0.995829  0.064460  0.0  1.0  1.0  1.0  1.0  
5                  4677.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  
6                  3036.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  
7                  7345.0  0.001361  0.036875  0.0  0.0  0.0  0.0  1.0  

[8 rows x 288 columns]

### 3.- Elegir Número de cluster

Estime mediante la **regla del codo**, el número de cluster apropiados para el caso.
Para efectos prácticos, eliga la siguiente secuencia como número de clusters a comparar:

$$[5, 10, 20, 30, 50, 75, 100, 200, 300]$$

Una ve realizado el gráfico, saque sus propias conclusiones del caso.


In [ ]:
Nc = [5,10,20,30,50,75,100,200,300]
kmeans = [KMeans(n_clusters=i) for i in Nc]
score = [kmeans[i].fit(df_procesado).inertia_ for i in range(len(kmeans))]

df_Elbow = pd.DataFrame({'Numero de Clusters':Nc,
                        'Score':score})
df_Elbow

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8.5))
plt.title('Elbow Curve')
sns.lineplot(x="Numero de Clusters",
             y="Score",
            data=df_Elbow)
sns.scatterplot(x="Numero de Clusters",
             y="Score",
             data=df_Elbow)
plt.show()

Viendo el grafico, podemos ver que con 100 clusters la grafica se empieza a estabilizar, por lo que ese seria un numero adecuado.

## II.- Problema 02

<img src="https://live.staticflickr.com/7866/47075467621_85ab810139_c.jpg" align="center"/>

Para el conjunto de datos de **Iris**, se pide realizar una reducción de dimensionalidad ocupando las técnicas de PCA y TSNE (vistas en clases). 

El objetivo es aplicar ambos algoritmos de la siguiente manera:

* Análisis detallado algoritma PCA (tablas, gráficos, etc.)
* Análisis detallado algoritma TSNE (tablas, gráficos, etc.)
* Comparar ambos algoritmos (conclusiones del caso)

In [20]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [21]:
dataset = load_iris()
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
target = 'species'

iris = pd.DataFrame(
    dataset.data,
    columns=features)

iris[target] = dataset.target
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
